Imports

In [1]:
from nb201 import NB201Benchmark
import numpy as np
from warmstart.utils_templates import FullTemplate
import ConfigSpace as CS
from ConfigSpace import Configuration
from transformers import AutoTokenizer, AutoModelForCausalLM
import ollama
import torchvision
from exp_baselines.bayesmark.data import ProblemType
import ast
from llambo.llambo import LLAMBO


/Users/e.lushtaku/anaconda3/envs/llambo/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load NB201 Benchmark

In [2]:
b = NB201Benchmark(path="./nb201.pkl", dataset='cifar10')
cs = b.get_configuration_space()
config = cs.sample_configuration()  # samples a configuration uniformly at random

print(cs)
print("Numpy representation: ", config.get_array())
print("Dict representation: ", config.get_dictionary())

#configuration from a dict
new_config = Configuration(cs, values=config.get_dictionary())
print(new_config)

y, cost = b.objective_function(config)
print("Test error: %f %%" % y)
print("Runtime %f s" % cost)

Configuration space object:
  Hyperparameters:
    op_0_to_1, Type: Categorical, Choices: {none, skip_connect, avg_pool_3x3, nor_conv_1x1, nor_conv_3x3}, Default: none
    op_0_to_2, Type: Categorical, Choices: {none, skip_connect, avg_pool_3x3, nor_conv_1x1, nor_conv_3x3}, Default: none
    op_0_to_3, Type: Categorical, Choices: {none, skip_connect, avg_pool_3x3, nor_conv_1x1, nor_conv_3x3}, Default: none
    op_1_to_2, Type: Categorical, Choices: {none, skip_connect, avg_pool_3x3, nor_conv_1x1, nor_conv_3x3}, Default: none
    op_1_to_3, Type: Categorical, Choices: {none, skip_connect, avg_pool_3x3, nor_conv_1x1, nor_conv_3x3}, Default: none
    op_2_to_3, Type: Categorical, Choices: {none, skip_connect, avg_pool_3x3, nor_conv_1x1, nor_conv_3x3}, Default: none

Numpy representation:  [2. 2. 0. 0. 4. 4.]
Dict representation:  {'op_0_to_1': 'avg_pool_3x3', 'op_0_to_2': 'avg_pool_3x3', 'op_0_to_3': 'none', 'op_1_to_2': 'none', 'op_1_to_3': 'nor_conv_3x3', 'op_2_to_3': 'nor_conv_3x3'}
Co

/var/folders/b0/nhhvw1x54fggv0t8nzr9s4xm0000gp/T/ipykernel_57299/2721149981.py:7: DeprecationWarning: `Configuration` act's like a dictionary. Please use `dict(config)` instead of `get_dictionary` if you explicitly need a `dict`
  print("Dict representation: ", config.get_dictionary())
/var/folders/b0/nhhvw1x54fggv0t8nzr9s4xm0000gp/T/ipykernel_57299/2721149981.py:10: DeprecationWarning: `Configuration` act's like a dictionary. Please use `dict(config)` instead of `get_dictionary` if you explicitly need a `dict`
  new_config = Configuration(cs, values=config.get_dictionary())


Arguments for LLAMBO

In [3]:
task_context = {
    'model': 'CNN',
    'task': 'classification',
    'tot_feats': 32 * 32 * 3,
    'cat_feats': 0,
    'num_feat': 32 * 32 * 3,
    'n_classes': 10,
    'metric': 'loss',
    'lower_is_better': True,
    'num_samples': 50000,
    'hyperparameter_constraints': {
        'op_0_to_1': ['categorical', None, ["none", "skip_connect", "avg_pool_3x3", "nor_conv_1x1", "nor_conv_3x3"]],
        # [type, transform, [min_value, max_value]]
        'op_0_to_2': ['categorical', None, ["none", "skip_connect", "avg_pool_3x3", "nor_conv_1x1", "nor_conv_3x3"]],
        'op_0_to_3': ['categorical', None, ["none", "skip_connect", "avg_pool_3x3", "nor_conv_1x1", "nor_conv_3x3"]],
        'op_1_to_2': ['categorical', None, ["none", "skip_connect", "avg_pool_3x3", "nor_conv_1x1", "nor_conv_3x3"]],
        'op_1_to_3': ['categorical', None, ["none", "skip_connect", "avg_pool_3x3", "nor_conv_1x1", "nor_conv_3x3"]],
        'op_2_to_3': ['categorical', None, ["none", "skip_connect", "avg_pool_3x3", "nor_conv_1x1", "nor_conv_3x3"]]
    }
}


def init_f():
    return


def eval_point(config):
    new_config = Configuration(b.get_configuration_space(), values=config)
    res = b.objective_function(new_config)
    res_dict = {
        "score": res[0],
        "train_time": res[1]
    }
    return config, res_dict

Huggingface

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it").to("cuda")

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2b-it.
401 Client Error. (Request ID: Root=1-66928f29-0873989f68d84519150649e5;b2bedcc8-cd62-4d94-9ca4-f25105f825ef)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must be authenticated to access it.

In [ ]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_length=512)
print(tokenizer.decode(outputs[0]))

Ollama

In [5]:
chat_engine = "llama3"
model = ollama.pull(chat_engine)
response = ollama.chat(model="llama3", messages=[{'role': 'user', 'content': 'Why is the sky blue?'}])
print(response)
ollama.list()

{'model': 'llama3', 'created_at': '2024-07-13T14:31:00.174974Z', 'message': {'role': 'assistant', 'content': "The short answer is that the sky appears blue because of a phenomenon called scattering. Here's what happens:\n\n1. **Sunlight enters Earth's atmosphere**: When the sun emits light, it travels through space and into our atmosphere.\n2. **Light interacts with tiny molecules**: As sunlight enters the atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of visible light.\n3. **Scattering occurs**: When light hits these molecules, they scatter the shorter wavelengths more effectively. This is because the molecules are more sensitive to the higher-energy, blue-end of the visible spectrum.\n4. **Blue light is scattered in all directions**: As a result of this scattering, the blue light is distributed evenly throughout the atmosphere, reaching our eyes from all parts of the sky.\n5. **Other wavelengt

{'models': [{'name': 'llama3:latest',
   'model': 'llama3:latest',
   'modified_at': '2024-07-13T16:30:51.271195723+02:00',
   'size': 4661224676,
   'digest': '365c0bd3c000a25d28ddbf732fe1c6add414de7275464c4e4d1c3b5fcb5d8ad1',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '8.0B',
    'quantization_level': 'Q4_0'}}]}

In [6]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True)
def fetch_statistics(dict, dataset):
    images = dataset.data
    labels = dataset.targets

    images_np = np.array(images)
    labels_np = np.array(labels)

    pixel_mean = np.mean(images_np / 255.)
    pixel_std = np.std(images_np / 255.)

    class_counts = np.bincount(labels_np)
    class_distribution = class_counts / len(labels_np)

    dict['pixel_mean'] = pixel_mean
    dict['pixel_std'] = pixel_std
    dict['class_distribution'] = class_distribution.tolist()

    return dict


task_context = fetch_statistics(task_context, trainset)

Files already downloaded and verified


Warmstart

In [7]:
config = "No_Context"
metric = "acc"
NUM_SEEDS = 10
problem_type = ProblemType.clf


def extract_configs_from_response(response):
    content = response['message']['content']
    start = content.find("[")
    end = content.rfind("]") + 1
    list_str = content[start:end]
    configurations = ast.literal_eval(list_str)
    return configurations


def is_dict_valid_in_config_space(d, config_space):
    try:
        # Attempt to create a Configuration object with the given dictionary and config space
        config = CS.Configuration(config_space, values=d)
        return True
    except:
        # Return False if the dictionary is not valid
        return False
    # Function to check if all dictionaries in a list are valid in the given configuration space


def check_all_list(parsed_dicts, config_space):
    for idx, d in enumerate(parsed_dicts):
        if not is_dict_valid_in_config_space(d, config_space):
            return False
    return True


def obtain_all_list_valid(resp, config_space):
    if check_all_list(resp, config_space):
        return resp
    print("fail")


def generate_init_conf(n_samples):
    template_object = FullTemplate(context=config, provide_ranges=True)
    input_prompt = template_object.add_context(config_space=cs, num_recommendation=n_samples, task_dict=task_context)
    response = ollama.chat(model="llama3", messages=[{'role': 'user', 'content': input_prompt}])
    configs = extract_configs_from_response(response)
    return obtain_all_list_valid(configs, cs)

#print(generate_init_conf(3))

Llambo

In [8]:
llambo = LLAMBO(task_context, sm_mode='discriminative', n_candidates=10, n_templates=2, n_gens=10,
                alpha=0.1, n_initial_samples=5, n_trials=4,
                init_f=generate_init_conf,
                bbox_eval_f=eval_point,
                chat_engine="llama3")
llambo.seed = 0

# run optimization
configs, fvals = llambo.optimize(test_metric="score")

[Search settings]: 
	n_candidates: 10, n_templates: 2, n_gens: 10, 
	alpha: 0.1, n_initial_samples: 5, n_trials: 4, 
	using warping: False, ablation: None, shuffle_features: False
[Task]: 
	task type: classification, sm: discriminative, lower is better: True
Hyperparameter search space: 
{'op_0_to_1': ['categorical',
               None,
               ['none',
                'skip_connect',
                'avg_pool_3x3',
                'nor_conv_1x1',
                'nor_conv_3x3']],
 'op_0_to_2': ['categorical',
               None,
               ['none',
                'skip_connect',
                'avg_pool_3x3',
                'nor_conv_1x1',
                'nor_conv_3x3']],
 'op_0_to_3': ['categorical',
               None,
               ['none',
                'skip_connect',
                'avg_pool_3x3',
                'nor_conv_1x1',
                'nor_conv_3x3']],
 'op_1_to_2': ['categorical',
               None,
               ['none',
                'skip

/Users/e.lushtaku/E/Personal/MasterStudies-UniversityOfFreiburg/SecondSemester/DeepLearningLab/Project/ModifiedProject/llambo/discriminative_sm.py:144: RuntimeWarning: Mean of empty slice
  y_mean = np.nanmean(all_preds, axis=1)
/Users/e.lushtaku/anaconda3/envs/llambo/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
